# Guest House - Hard

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app13-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@460d05be

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@7cb484c4

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val booking = hiveCxt.table("sqlzoo.booking")
val guest = hiveCxt.table("sqlzoo.guest")
val room = hiveCxt.table("sqlzoo.room")
val rate = hiveCxt.table("sqlzoo.rate")
val extra = hiveCxt.table("sqlzoo.extra")

booking: DataFrame = [booking_id: int, booking_date: string ... 6 more fields]
guest: DataFrame = [id: int, first_name: string ... 2 more fields]
room: DataFrame = [id: int, room_type: string ... 1 more field]
rate: DataFrame = [room_type: string, occupancy: int ... 1 more field]
extra: DataFrame = [extra_id: int, booking_id: int ... 2 more fields]

## 11.
Coincidence. Have two guests with the same surname ever stayed in the hotel on the evening? Show the last name and both first names. Do not include duplicates.

```
+-----------+------------+-------------+
| last_name | first_name | first_name  |
+-----------+------------+-------------+
| Davies    | Philip     | David T. C. |
| Evans     | Graham     | Mr Nigel    |
| Howarth   | Mr George  | Sir Gerald  |
| Jones     | Susan Elan | Mr Marcus   |
| Lewis     | Clive      | Dr Julian   |
| McDonnell | John       | Dr Alasdair |
+-----------+------------+-------------+
```

In [5]:
val t = (guest.join(booking, (guest("id")===booking("guest_id")))
     .select("last_name", "first_name", "booking_date", "nights", "id")
     .withColumn("checkout", date_add(col("booking_date"), col("nights")-lit(1))))

val a = (t.join(t.alias("t1"), "last_name", joinType="full")
     .toDF("last_name", "first_name", "booking_date", "nights", "id", "checkout",
           "first_name1", "booking_date1", "nights1", "id1", "checkout1")
     .filter(col("first_name") !== col("first_name1")))

(a.filter((((col("booking_date") >= col("booking_date1")) && 
            (col("booking_date") <= col("checkout1"))) ||
          ((col("booking_date1") >= col("booking_date")) && 
            (col("booking_date1") <= col("checkout")))) &&
          (col("id") >= col("id1")))
 .select("last_name", "first_name1", "first_name")
 .distinct()
 .orderBy("last_name")
 .showHTML())

last_name,first_name1,first_name
Davies,Philip,David T. C.
Evans,Graham,Mr Nigel
Howarth,Mr George,Sir Gerald
Jones,Susan Elan,Mr Marcus
Lewis,Clive,Dr Julian
McDonnell,John,Dr Alasdair


t: DataFrame = [last_name: string, first_name: string ... 4 more fields]
a: Dataset[Row] = [last_name: string, first_name: string ... 9 more fields]

## 12.
Check out per floor. The first digit of the room number indicates the floor – e.g. room 201 is on the 2nd floor. For each day of the week beginning 2016-11-14 show how many rooms are being vacated that day by floor number. Show all days in the correct order.

```
+------------+-----+-----+-----+
| i          | 1st | 2nd | 3rd |
+------------+-----+-----+-----+
| 2016-11-14 |   5 |   3 |   4 |
| 2016-11-15 |   6 |   4 |   1 |
| 2016-11-16 |   2 |   2 |   4 |
| 2016-11-17 |   5 |   3 |   6 |
| 2016-11-18 |   2 |   3 |   2 |
| 2016-11-19 |   5 |   5 |   1 |
| 2016-11-20 |   2 |   2 |   2 |
+------------+-----+-----+-----+
```

In [6]:
(booking
 .withColumn("checkout", date_add(col("booking_date"), col("nights")))
 .withColumn("floor", substring(col("room_no"), 1, 1))
 .filter(col("checkout").between("2016-11-14", "2016-11-20"))
 .na.replace("floor", Map("1" -> "1st", "2" -> "2nd", "3" -> "3rd", "4" -> "4th"))
 .groupBy("checkout")
 .pivot("floor")
 .count()
 .orderBy("checkout")
 .showHTML())

checkout,1st,2nd,3rd
2016-11-14,5,3,4
2016-11-15,6,4,1
2016-11-16,2,2,4
2016-11-17,5,3,6
2016-11-18,2,3,2
2016-11-19,5,5,1
2016-11-20,2,2,2


## 13.
Free rooms? List the rooms that are free on the day 25th Nov 2016.

```
+-----+
| id  |
+-----+
| 207 |
| 210 |
| 304 |
+-----+
```

In [7]:
(room.join(
    booking
    .withColumn("checkout", date_add(col("booking_date"), col("nights")-1))
    .filter((col("booking_date") <= "2016-11-25") && 
            (col("checkout") >= "2016-11-25")),
    (booking("room_no")===room("id")), joinType="left_anti")
 .select("id")
 .showHTML())

id
207
210
304


## 14.
Single room for three nights required. A customer wants a single room for three consecutive nights. Find the first available date in December 2016.

```
+-----+------------+
| id  | MIN(i)     |
+-----+------------+
| 201 | 2016-12-11 |
+-----+------------+
```

In [8]:
(booking
 .withColumn("checkout", date_add(col("booking_date"), col("nights")))
 .join(room.filter(col("room_type")==="single"), 
       (booking("room_no")===room("id")), joinType="right")
 .filter(((col("checkout") >= "2016-12-01") && 
          (col("booking_date") <= "2016-12-31")) || 
         (col("checkout").isNull))
 .withColumn("next_booking", lag("booking_date", 1, "2017-01-01").over(
     Window.partitionBy("room_no").orderBy("room_no")))
 .withColumn("diff", (col("next_booking")-col("checkout")).cast("integer"))
 .filter(col("diff") >= 3)
 .select("room_no", "checkout")
 .orderBy("checkout")
 .limit(1)
 .showHTML())

room_no,checkout
201,2016-12-11


## 15.
Gross income by week. Money is collected from guests when they leave. For each Thursday in November and December 2016, show the total amount of money collected from the previous Friday to that day, inclusive.

```
+------------+---------------+
| Thursday   | weekly_income |
+------------+---------------+
| 2016-11-03 |          0.00 |
| 2016-11-10 |      12608.94 |
| 2016-11-17 |      13552.56 |
| 2016-11-24 |      12929.69 |
| 2016-12-01 |      11685.14 |
| 2016-12-08 |      13093.79 |
| 2016-12-15 |       8975.87 |
| 2016-12-22 |       1395.77 |
| 2016-12-29 |          0.00 |
| 2017-01-05 |          0.00 |
+------------+---------------+
```

In [12]:
// generate Thursdays
import java.time.LocalDate
import java.time.DayOfWeek
import java.time.temporal.TemporalAdjusters

def getThursdays(startDate: LocalDate, endDate: LocalDate): List[LocalDate] = {
  val thursdays = scala.collection.mutable.ListBuffer.empty[LocalDate]
  var nextThursday = startDate.`with`(TemporalAdjusters.nextOrSame(DayOfWeek.THURSDAY))

  while (!nextThursday.isAfter(endDate)) {
    thursdays += nextThursday
    nextThursday = nextThursday.plusWeeks(1)
  }

  thursdays.toList
}

val thursdays = spark.createDataset(
    getThursdays(LocalDate.of(2016, 11, 1), LocalDate.of(2016, 12, 31))).toDF("Thursday")

thursdays.show()

+----------+
|  Thursday|
+----------+
|2016-11-03|
|2016-11-10|
|2016-11-17|
|2016-11-24|
|2016-12-01|
|2016-12-08|
|2016-12-15|
|2016-12-22|
|2016-12-29|
+----------+



import java.time.LocalDate

import java.time.DayOfWeek

import java.time.temporal.TemporalAdjusters


defined function getThursdays
thursdays: DataFrame = [Thursday: date]

In [10]:
(booking
 .join(rate, (booking("occupants")===rate("occupancy")) && 
             (booking("room_type_requested")===rate("room_type")), 
       joinType="left")
 .withColumn("income", col("amount") * col("nights"))
 .select("booking_date", "nights", "income")
 .union(booking
        .join(extra, "booking_id", joinType="left")
        .withColumn("income", col("amount"))
        .select("booking_date", "nights", "income"))
 .withColumn("checkout", date_add(col("booking_date"), col("nights")))
 .withColumn("wkd", dayofweek(col("checkout")))
 .withColumn("Thursday", when(
     col("wkd")<=5, date_add(col("checkout"), lit(5)-col("wkd")))
     .otherwise(date_add(col("checkout"), lit(12)-col("wkd"))))
 .join(thursdays, "Thursday", joinType="right")
 .groupBy("Thursday")
 .agg(round(sum("income"), 2).alias("weekly_income"))
 .na.fill(0, Array("weekly_income"))
 .orderBy("Thursday")
 .showHTML())

Thursday,weekly_income
2016-11-03,0.0
2016-11-10,12608.94
2016-11-17,13552.56
2016-11-24,12929.69
2016-12-01,11685.14
2016-12-08,13093.79
2016-12-15,8975.87
2016-12-22,1395.77
2016-12-29,0.0


In [11]:
spark.stop()